<a href="https://colab.research.google.com/github/ruanrabbets-tech/NH3-Decomposition-Workflow/blob/main/Modified_Step_2_(Create_and_relax_slab_and_adsorbate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ase==3.23.0

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# install quantum espresso from conda
!conda install conda-forge::qe

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Below imports necessary functions
from IPython.display import HTML, Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

from ase import Atoms
from ase.build import make_supercell, bulk, fcc111, molecule, add_adsorbate
from ase.io import read, write
from ase.visualize import view

import os
os.makedirs("output", exist_ok=True)

import numpy as np


# Below are functions necessary for viewing the structures
def view_x3d(atoms, idx=0):
    if isinstance(atoms[0], Atoms):
        # Assume this is a trajectory or struct list
        if (len(atoms) <= idx):
                print(f"The specified index exceeds the length of the trajectory. The length of the trajectory is {len(atoms)}.")
        return view(atoms[idx], viewer="x3d")
    else:
        return view(atoms, viewer="x3d")


def view_ase_atoms(atoms, rotation="0x,0y,0z", figsize=(4, 4), title="", scale=100):
    fig, ax = plt.subplots(figsize=figsize)
    write("output/tmp.png", atoms, rotation=rotation, scale=scale)
    img = mpimg.imread('output/tmp.png')
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
    plt.show()
    os.remove('output/tmp.png')
    return


def traj_to_apng(traj, rotation='30x,30y,30z'):
    imgs = []
    for atom in traj:
        supercell = make_supercell(atom, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        write('output/tmp.png', supercell, rotation=rotation, show_unit_cell=2)
        img = mpimg.imread('output/tmp.png')
        imgs.append(img)
    os.remove('output/tmp.png')

    fig, ax = plt.subplots()

    def update(frame):
        img = imgs[frame]
        ax.clear()
        ax.imshow(img)
        return []

    ani = FuncAnimation(fig, update, frames=len(imgs), blit=True)
    plt.close()
    return HTML(ani.to_jshtml())

### Below is a function I created for running quantum expresso.  I'm using my  **3rd version** because it saves the output straight to my google drive.

In [ ]:
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import BFGS
import shutil
import os

def run_qe_calculation(structure, calculation_type, pseudopotentials, calculation_label="Not_given"):
    """
    Runs a Quantum ESPRESSO calculation using ASE.

    Parameters:
        calculation_type (str): e.g., 'relax', 'scf', etc.
        pseudopotentials (dict): Mapping of elements to UPF files.
        structure (ase.Atoms): ASE Atoms object to be relaxed or calculated.

    Returns:
        float: Potential energy of the relaxed/final structure.
    """

    # Mount Google Drive ONLY if it's not already mounted
    from google.colab import drive
    try:
        drive.mount('/content/drive')
    except ValueError:
        pass  # Ignore if already mounted

    # Set the folder path on Google Drive (where I want my results to be saved)
    drive_folder = '/content/drive/MyDrive/My Results'  # Ruan and adjust the folder path as needed
    calculation_folder = os.path.join(drive_folder, calculation_label)

    # Create the folder if it does not exist
    os.makedirs(calculation_folder, exist_ok=True)

    # Set up Quantum ESPRESSO input
    input_data = {
        'control': {
            'calculation': calculation_type,
            'verbosity': 'high',
            'pseudo_dir': os.path.abspath('./'),  # Use os.path.abspath to ensure absolute path
            'restart_mode': 'from_scratch',
            'tstress': True,
            'tprnfor': True,
            'outdir': calculation_folder,  # Save QE native outputs here
            'max_seconds': 36000
        },
        'system': {
            'ecutwfc': 30,
            'ecutrho': 240,
            'occupations': 'smearing',
            'smearing': 'mv', # Use smearing of "mp" or "mv" when dealing with metals and use "gaussian" when dealing with molecules.
            'degauss': 0.05,  # when working with adsorbate change to 0.001
            'nspin': 2,
            'starting_magnetization(1)': 0.5
        },
        'electrons': {
            'diagonalization': 'cg',        # was 'david'
            'mixing_mode':     'local-TF',  # was 'plain'
            'mixing_beta':     0.2,         # was 0.7
            'mixing_ndim':     12,          # new: extra history vectors
            'electron_maxstep': 400,        # new: allow more SCF iterations
            'conv_thr':        1.0e-5
    },
}

    profile = EspressoProfile(command='/usr/local/bin/pw.x', pseudo_dir='./')  # or os.path.abspath('./') if pseudo_dir is relative

    calc = Espresso(
        profile=profile,
        input_data=input_data,
        pseudopotentials=pseudopotentials,
        kpts=(2, 2, 1),               # Note from Prof. Cecil: if slab dimensions is (4,4,1) use k-point of "2,2,1", or use a gamma k-point i.e "1,1,1". We can use 4,4,4 for unit cells.
        directory=calculation_folder  # Save the output in the specified directory
    )

    structure.calc = calc

    # Run calculation
    energy = structure.get_potential_energy()

    #*************** Below code is an attempt to put all outputs into a specific folder *******########
    # Save energy summary to a text file
    output_file = os.path.join(calculation_folder, f'{calculation_label}_output.txt')
    with open(output_file, 'w') as f:
        f.write(f'Energy of the relaxed structure: {energy} eV\n')

    # Save final structure as CIF
    cif_path = os.path.join(calculation_folder, f'{calculation_label}_relaxed_structure.cif')
    write(cif_path, structure)  # Save relaxed structure

    # Redundantly ensure all generated files are in the Google Drive folder
    try:
        # Print source and destination to debug the issue
        print(f"Source directory: {calc.directory}")
        print(f"Destination directory: {calculation_folder}")

        # Copy files manually, avoid using copytree in case of special characters to avoid errors
        for item in os.listdir(calc.directory):
            s = os.path.join(calc.directory, item)
            d = os.path.join(calculation_folder, item)
            if os.path.isdir(s):
                shutil.copytree(s, d, dirs_exist_ok=True)  # If it's a directory, copy it recursively
            else:
                shutil.copy2(s, d)  # If it's a file, copy it

    except Exception as e:
        print(f"Error during file copy: {e}")

    return energy

### Below code generates a pseudopotential dictionary from all the available pseusopotential files available in the directory

In [ ]:
def generate_pseudopotential_dict(pseudo_dir='./'):
    """
    Automatically generates a pseudopotential dictionary by scanning a directory for .UPF files.

    Args:
        pseudo_dir (str): Path to the directory containing UPF files.

    Returns:
        dict: Dictionary in the form {element: filename}
    """
    pp_dict = {}
    for file in os.listdir(pseudo_dir):
        if file.endswith(".UPF"):
            element = file.split('.')[0]  # assumes element is the first part of the filename
            pp_dict[element] = file
    return pp_dict

### Below function I created helps me freeze atoms in the desired layers:

In [ ]:
def freeze(structure, no_of_layers_from_bottom_to_be_frozen):   #note "structure" refers to tht bulk structure in which I want to freeze some atoms.
  # Below gets the cordinate of the atoms in the structure
  atoms = structure
  import pandas as pd #importing this in case I fogot to import it earlier
  df = pd.DataFrame({
    "x": atoms.positions[:, 0],
    "y": atoms.positions[:, 1],
    "z": atoms.positions[:, 2],
    "symbol": atoms.symbols,
  })
  #Below isolates the z cordinates of all the atoms into a list
  z_list = df["z"].tolist()
  unique_z_values = list(set(z_list))  # removes duplicates
  sorted_z = sorted(unique_z_values, reverse=False)  # Sort from smallest to largest
  #identify the tresh of the z cordinates of the atoms to be frozen
  thresh =  (sorted_z[no_of_layers_from_bottom_to_be_frozen] + sorted_z[no_of_layers_from_bottom_to_be_frozen-1])/2
  # print(thresh) #************************************************************************************************************************************** For troubleshooting
  #below applies the constraint to the atoms in the layers I deliniated with the treshold
  from ase.constraints import FixAtoms   # In case I forget to call it earlier
  constraint = FixAtoms(mask=structure.positions[:, 2] < thresh)
  structure.set_constraint(constraint)
  return structure



### I  run the calculation from below codes

In [ ]:
#step 1: Upload the necessary Pseudopotential files
from google.colab import files
uploaded = files.upload()

Saving Fe.pbesol-spn-kjpaw_psl.1.0.0.UPF to Fe.pbesol-spn-kjpaw_psl.1.0.0 (2).UPF


In [ ]:
#Step 2: Generate a pseudo dictionary with my function
pseudo_dict = generate_pseudopotential_dict(pseudo_dir='./')
print(pseudo_dict)

{'Fe': 'Fe.pbesol-spn-kjpaw_psl.1.0.0.UPF'}


### For Loop

In [ ]:
# Making 3x3x4 Structures
element_list = []
Energy_absorbed_slab = []
calc_duration = []

In [ ]:
from ase.build import bulk, fcc111, bcc111, surface
import pandas as pd
import time

First_row = ["Sc", "Ti", "V", "Cr", "Fe", "Co", "Ni", "Cu", "Zn"]
Crystal_structure = ["hcp", "hcp", "bcc", "bcc", "bcc", "hcp", "fcc", "fcc", "hcp"]
Lattice = [3.31, 2.95, 3.03, 2.88, 2.87, 2.51, 3.52, 3.61, 2.66]
Lattice_HCP = [5.27, 4.68, 0, 0, 0, 4.07, 0, 0, 4.95]

for element in First_row:
    Element = element
    print(element)

    try:

      if Crystal_structure[First_row.index(element)] == "hcp":
          bulk2 = bulk(Element, crystalstructure='hcp', a=Lattice[First_row.index(element)], c=Lattice_HCP[First_row.index(element)])
          surface2 = surface(bulk2, (0, 0, 1), layers=2, vacuum=15.0)
          slab_unfrozen2 = surface2 * (3,3,1)
          slab2 = freeze(slab_unfrozen2, 2) #freeze the bottom 2 layers
          # print("hcp_done")

      elif Crystal_structure[First_row.index(element)] == "bcc":
          slab_unfrozen2 = bcc111(Element, size=(3, 3, 4), vacuum=15.0, a=Lattice[First_row.index(element)])
          slab2 = freeze(slab_unfrozen2, 2) #freeze the bottom 2 layers
          # print("bcc_done")

      elif Crystal_structure[First_row.index(element)] == "fcc":
          slab_unfrozen2 = fcc111(Element, size=(3, 3, 4), vacuum=15.0, a=Lattice[First_row.index(element)])
          slab2 = freeze(slab_unfrozen2, 2) #freeze the bottom 2 layers
          # print("fcc_done")

      # Run QE calculation and keep time
      start = time.time()
      Calc_type = "relax"
      calc_label = Element + "_(3,3,4)" + Calc_type
      E = run_qe_calculation(slab2, Calc_type, pseudo_dict, calc_label)
      Energy_absorbed_slab.append(E)
      end = time.time()

      #Below code gets the duration of the calculation
      duration = end - start
      hours = round(duration // 3600)
      minutes = round((duration % 3600) // 60)
      seconds = duration # Assign the total duration to seconds
      remaining_seconds = round(seconds % 60)
      print("Relaxation finished in", hours ,  'hours', minutes, "minutes, and",remaining_seconds, "seconds" )
      to_STR = str(hours) + "h :" + str(minutes) + "m :" + str(remaining_seconds) + "s" #converting the time the calc took into string, so it can be appended
      calc_duration.append(to_STR)                      #********************************************************************append duration of the calculation

      element_list.append(element)

    except Exception as e:
        print(f"{element} failed with error: {e}")
        Energy_absorbed_slab.append("failed")
        element_list.append("failed")
        calc_duration.append("failed")

    df = pd.DataFrame({
        'First_Row': element_list,
        'Energy': Energy,
        'Calcualtion Duration': calc_duration
    })

    excel_path = "/home/doduma/quantum_expresso_result_outputs/Excel_sheet_outputs/Result_excel_sheet.xlsx" #************************************Modify the path
    sheet_name = '3x3x4 Slab - First Row'  #****************************************************************************************** Change this to your desired sheet name

    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'First_Row': element_list,
    'Energy': Energy
})

excel_path = '/content/drive/Shareddrives/CSC python works/Quantum_Espresso_Results/Lattice.xlsx'
sheet_name = '3x3x2 Slab - First Row'  # Change this to your desired sheet name

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

# For individual calculations

In [ ]:
#Step 3: create the bulk structrue
from ase.build import bulk, surface
Element = 'Fe'
slab_unfrozen = fcc111(Element, size=(3, 3, 4), vacuum=15.0, a=2.48549290886134)
slab = freeze(slab_unfrozen, 2) #freeze the bottom 2 layers

#view the bulk structure
view_x3d(slab)

In [ ]:
#Step 4: carry out a relax on the unit cell
import time
start = time.time()
Calc_type = "relax"
calc_label = Element + "_(3,3,4)" + Calc_type
E = run_qe_calculation(slab, Calc_type, pseudo_dict, calc_label)
end = time.time()


#Below code gets the duration of the calculation
duration = end - start
hours = duration // 3600
minutes = (duration % 3600) // 60
seconds = duration # Assign the total duration to seconds
remaining_seconds = seconds % 60
print("Relaxation finished in", hours ,  'hours', minutes, "minutes, and",remaining_seconds, "seconds" )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


StopIteration: 

In [ ]:
#print the energy of the bulk surface
print('The energy of the bulk surface is:',E)